# Bab 8 Backpropagation (2)

## Praktikum

### a) Fungsi One-hot Encoding dan Decoding

In [1]:
import numpy as np
def onehot_enc(lbl, min_val=0):
  mi = min(lbl)
  enc = np.full((len(lbl), max(lbl) - mi + 1), min_val, np.int8)
  for i, x in enumerate(lbl):
    enc[i, x - mi] = 1
  return enc

def onehot_dec(enc, mi=0):
  return [np.argmax(e) + mi for e in enc]

### b) Fungsi Aktivasi Sigmoid dan derivatifnya

In [2]:
def sig(X):
  return [1 / (1 + np.exp(-x)) for x in X]

def sigd(X):
  output = []
  for i, x in enumerate(X):
    s = sig([x])[0]
    output.append(s * (1 - s))
  return output

### c) Fungsi *Training* Backpropagation

Tulis kode ke dalam *cell* di bawah ini:

In [7]:
import numpy as np

def bp_fit(X, target, layer_conf, max_epoch, max_error=.1, learn_rate=.1, print_per_epoch=100):
  np.random.seed(1)
  nin = [np.empty(i) for i in layer_conf]
  n = [np.empty(j + 1) if i < len(layer_conf) - 1 else np.empty(j) for i, j in enumerate(layer_conf)]
  w = np.array([np.random.rand(layer_conf[i] + 1, layer_conf[i + 1]) for i in range(len(layer_conf) - 1)], dtype=object)
  dw = [np.empty((layer_conf[i] + 1, layer_conf[i + 1])) for i in range(len(layer_conf) - 1)]
  d = [np.empty(s) for s in layer_conf[1:]]
  din = [np.empty(s) for s in layer_conf[1:-1]]
  epoch = 0
  mse = 1

  for i in range(0, len(n)-1):
    n[i][-1] = 1

  while (max_epoch == -1 or epoch < max_epoch) and mse > max_error:
    epoch += 1
    mse = 0

    for r in range(len(X)):
      n[0][:-1] = X[r]

      for L in range(1, len(layer_conf)):
        nin[L] = np.dot(n[L-1], w[L-1])
        n[L][:len(nin[L])] = sig(nin[L])
      e = target[r] - n[-1]
      mse += sum(e ** 2)
      d[-1] = e * sigd(nin[-1])
      dw[-1] = learn_rate * d[-1] * n[-2].reshape((-1, 1))
      for L in range(len(layer_conf) - 1, 1, -1):
        din[L-2] = np.dot(d[L-1], np.transpose(w[L-1][:-1]))
        d[L-2] = din[L-2] * np.array(sigd(nin[L-1]))
        dw[L-2] = (learn_rate * d[L-2]) * n[L-2].reshape((-1, 1))
      w += dw
    mse /= len(X)
    if print_per_epoch > -1 and epoch % print_per_epoch == 0:
      print(f'Epoch {epoch}, MSE: {mse}')

  return w, epoch, mse

### d) Fungsi *Testing* Backpropagation

Tulis kode ke dalam *cell* di bawah ini:

In [4]:
def bp_predict(X, w):
  n = [np.empty(len(i)) for i in w]
  nin = [np.empty(len(i[0])) for i in w]
  predict = []
  n.append(np.empty(len(w[-1][0])))
  for x in X:
    n[0][:-1] = x
    for L in range(0, len(w)):
      nin[L] = np.dot(n[L], w[L])
      n[L + 1][:len(nin[L])] = sig(nin[L])
    predict.append(n[-1].copy())
  return predict

### e) Percobaan Klasifikasi Dataset Iris

![Iris Dataset](https://raw.githubusercontent.com/ApoorvRusia/SVM-classification-on-Iris-dataset/master/iris_types.jpg)

Tulis kode ke dalam *cell* di bawah ini:

In [8]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score

iris = datasets.load_iris()
X = minmax_scale(iris.data)
Y = onehot_enc(iris.target)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.3,random_state=1)
w, ep, mse = bp_fit(X_train, y_train, layer_conf=(4, 3, 3), learn_rate=.1, max_epoch=1000, max_error=.1, print_per_epoch=25)
print(f'Epochs: {ep}, MSE: {mse}')
predict = bp_predict(X_test, w)
predict = onehot_dec(predict)
y_test = onehot_dec(y_test)
accuracy = accuracy_score(predict, y_test)
print('Output:', predict)
print('True :', y_test)
print('Accuracy:', accuracy)

C:\Users\tridi\AppData\Local\Temp\ipykernel_11276\2865347377.py:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  w += dw


Epoch 25, MSE: 0.4573000553790559
Epoch 50, MSE: 0.321272689922169
Epoch 75, MSE: 0.26680034509393197
Epoch 100, MSE: 0.19045841193641888
Epoch 125, MSE: 0.1320606403281753
Epoch 150, MSE: 0.10002434429710474
Epochs: 151, MSE: 0.09910797309769233
Output: [0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1, 2, 0, 2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 2, 2, 1]
True : [0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 2, 2, 1]
Accuracy: 0.9777777777777777


In [62]:
import warnings
warnings.filterwarnings("ignore")

## Soal 1

In [63]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score

iris = datasets.load_iris()
X = minmax_scale(iris.data)
Y = onehot_enc(iris.target)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.3,random_state=1)
w, ep, mse = bp_fit(X_train, y_train, layer_conf=(4, 2, 3), learn_rate=.1, max_epoch=100, max_error=.5, print_per_epoch=25)
print(f'Epochs: {ep}, MSE: {mse}')

# clone w
w2 = np.array([i.copy() for i in w], dtype=object)

Epoch 25, MSE: 0.5587148548510855
Epochs: 29, MSE: 0.49569003845261594


In [64]:
# Testing dengan data testing
predict = bp_predict(X_test, w)
predict = onehot_dec(predict)
y_test = onehot_dec(y_test)
accuracy = accuracy_score(predict, y_test)
print('Output:', predict)
print('True :', y_test)
print('Accuracy:', accuracy)

Output: [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
True : [0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 2, 2, 1]
Accuracy: 0.28888888888888886


In [65]:
# Testing dengan data latih
predict = bp_predict(X_train, w2)
predict = onehot_dec(predict)
y_train = onehot_dec(y_train)
accuracy = accuracy_score(predict, y_train)
print('Output:', predict)
print('True :', y_train)
print('Accuracy:', accuracy)

Output: [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
True : [2, 0, 0, 0, 1, 0, 0, 2, 2, 2, 2, 2, 1, 2, 1, 0, 2, 2, 0, 0, 2, 0, 2, 2, 1, 1, 2, 2, 0, 1, 1, 2, 1, 2, 1, 0, 0, 0, 2, 0, 1, 2, 2, 0, 0, 1, 0, 2, 1, 2, 2, 1, 2, 2, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 2, 2, 2, 0, 0, 1, 0, 2, 0, 2, 2, 0, 2, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 2, 0, 0, 2, 1, 2, 1, 2, 2, 1, 2, 0]
Accuracy: 0.3523809523809524


## Soal 2

In [66]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score

iris = datasets.load_iris()
X = minmax_scale(iris.data)
Y = onehot_enc(iris.target)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.3,random_state=1)
w, ep, mse = bp_fit(X_train, y_train, layer_conf=(4, 25, 3), learn_rate=.1, max_epoch=10000, max_error=.01, print_per_epoch=25)
print(f'Epochs: {ep}, MSE: {mse}')

# clone w
w2 = np.array([i.copy() for i in w], dtype=object)

Epoch 25, MSE: 1.9997475602526067
Epoch 50, MSE: 1.9996839107720699
Epoch 75, MSE: 1.9995729256252905
Epoch 100, MSE: 1.9993234166160632
Epoch 125, MSE: 1.998004842894972
Epoch 150, MSE: 1.5034310711672474
Epoch 175, MSE: 0.9882734055300789
Epoch 200, MSE: 0.9358385629366138
Epoch 225, MSE: 0.8962403847767554
Epoch 250, MSE: 0.572011308818784
Epoch 275, MSE: 0.17189641558830482
Epoch 300, MSE: 0.13551945448567235
Epoch 325, MSE: 0.10976277201695427
Epoch 350, MSE: 0.09159154648169054
Epoch 375, MSE: 0.07929291362007071
Epoch 400, MSE: 0.07095077966556529
Epoch 425, MSE: 0.06511610472051664
Epoch 450, MSE: 0.06088086243803242
Epoch 475, MSE: 0.05769828099112584
Epoch 500, MSE: 0.05523313404391773
Epoch 525, MSE: 0.053273219995204825
Epoch 550, MSE: 0.051680019060663
Epoch 575, MSE: 0.050360624096748935
Epoch 600, MSE: 0.04925114278017034
Epoch 625, MSE: 0.04830653743243757
Epoch 650, MSE: 0.04749424790188583
Epoch 675, MSE: 0.0467901200965703
Epoch 700, MSE: 0.04617577407555435
Epoch 72

In [67]:
# Testing dengan data testing
predict = bp_predict(X_test, w)
predict = onehot_dec(predict)
y_test = onehot_dec(y_test)
accuracy = accuracy_score(predict, y_test)
print('Output:', predict)
print('True :', y_test)
print('Accuracy:', accuracy)

Output: [0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1, 2, 0, 2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 1, 2, 1]
True : [0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 2, 2, 1]
Accuracy: 0.9555555555555556


In [68]:
# Testing dengan data latih
predict = bp_predict(X_train, w2)
predict = onehot_dec(predict)
y_train = onehot_dec(y_train)
accuracy = accuracy_score(predict, y_train)
print('Output:', predict)
print('True :', y_train)
print('Accuracy:', accuracy)

Output: [2, 0, 0, 0, 1, 0, 0, 2, 2, 2, 2, 2, 1, 2, 1, 0, 2, 2, 0, 0, 2, 0, 2, 2, 1, 1, 2, 2, 0, 1, 1, 2, 1, 2, 1, 0, 0, 0, 2, 0, 2, 2, 2, 0, 0, 1, 0, 2, 1, 2, 2, 1, 2, 2, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 2, 2, 2, 0, 0, 1, 0, 2, 0, 2, 2, 0, 2, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 2, 0, 0, 2, 1, 2, 1, 2, 2, 1, 2, 0]
True : [2, 0, 0, 0, 1, 0, 0, 2, 2, 2, 2, 2, 1, 2, 1, 0, 2, 2, 0, 0, 2, 0, 2, 2, 1, 1, 2, 2, 0, 1, 1, 2, 1, 2, 1, 0, 0, 0, 2, 0, 1, 2, 2, 0, 0, 1, 0, 2, 1, 2, 2, 1, 2, 2, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 2, 2, 2, 0, 0, 1, 0, 2, 0, 2, 2, 0, 2, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 2, 0, 0, 2, 1, 2, 1, 2, 2, 1, 2, 0]
Accuracy: 0.9904761904761905


## Soal 3

In [69]:
hidden_neurons_value = [i for i in range(2, 11)]
hidden_neurons_value.remove(4)
max_epoch_value = [i for i in range(100, 1001, 100)]
max_error_value = [i/100 for i in range(1, 11)]

In [70]:
import pandas as pd

result_tables = []

In [71]:
# iterate over all possible combinations of hyperparameters
for hidden_neurons in hidden_neurons_value:
  for max_epoch in max_epoch_value:
    for max_error in max_error_value:
      # print hyperparameters combination
      print(f'hidden_neurons: {hidden_neurons}, max_epoch: {max_epoch}, max_error: {max_error}')

      w, ep, mse = bp_fit(X_train, y_train, layer_conf=(4, hidden_neurons, 3), learn_rate=.1, max_epoch=max_epoch, max_error=max_error, print_per_epoch=-1)

      # clone w
      w2 = np.array([i.copy() for i in w], dtype=object)

      # Testing dengan data testing
      predict = bp_predict(X_test, w)
      predict = onehot_dec(predict)
      y_test = onehot_dec(y_test)
      accuracy = accuracy_score(predict, y_test)

      # Testing dengan data latih
      predict = bp_predict(X_train, w2)
      predict = onehot_dec(predict)
      y_train = onehot_dec(y_train)
      accuracy2 = accuracy_score(predict, y_train)

      # insert the result into result_tables
      result_tables.append([hidden_neurons, max_epoch, max_error, accuracy, accuracy2])

hidden_neurons: 2, max_epoch: 100, max_error: 0.01
hidden_neurons: 2, max_epoch: 100, max_error: 0.02
hidden_neurons: 2, max_epoch: 100, max_error: 0.03
hidden_neurons: 2, max_epoch: 100, max_error: 0.04
hidden_neurons: 2, max_epoch: 100, max_error: 0.05
hidden_neurons: 2, max_epoch: 100, max_error: 0.06
hidden_neurons: 2, max_epoch: 100, max_error: 0.07
hidden_neurons: 2, max_epoch: 100, max_error: 0.08
hidden_neurons: 2, max_epoch: 100, max_error: 0.09
hidden_neurons: 2, max_epoch: 100, max_error: 0.1
hidden_neurons: 2, max_epoch: 200, max_error: 0.01
hidden_neurons: 2, max_epoch: 200, max_error: 0.02
hidden_neurons: 2, max_epoch: 200, max_error: 0.03
hidden_neurons: 2, max_epoch: 200, max_error: 0.04
hidden_neurons: 2, max_epoch: 200, max_error: 0.05
hidden_neurons: 2, max_epoch: 200, max_error: 0.06
hidden_neurons: 2, max_epoch: 200, max_error: 0.07
hidden_neurons: 2, max_epoch: 200, max_error: 0.08
hidden_neurons: 2, max_epoch: 200, max_error: 0.09
hidden_neurons: 2, max_epoch: 20

In [72]:
result_tables[500]

[8, 100, 0.01, 0.5777777777777777, 0.5523809523809524]

In [73]:
result_df = pd.DataFrame(result_tables, columns=['hidden_neurons', 'max_epoch', 'max_error', 'accuracy', 'accuracy2'])

result_df[(result_df['accuracy'] == 1) & (result_df['accuracy2'] == 1)]


,hidden_neurons,max_epoch,max_error,accuracy,accuracy2
1,2,100,0.02,1.0,1.0
2,2,100,0.03,1.0,1.0
3,2,100,0.04,1.0,1.0
4,2,100,0.05,1.0,1.0
5,2,100,0.06,1.0,1.0
...,...,...,...,...,...
295,5,1000,0.06,1.0,1.0
296,5,1000,0.07,1.0,1.0
297,5,1000,0.08,1.0,1.0
298,5,1000,0.09,1.0,1.0
